# 家用电器用户行为分析与事件识别

# 挖掘目标

- 根据热水器采集到的数据，划分一次完整的用水事件
- 在划分好的一次完整的用水事件中，识别出洗浴事件

## 分析方法与过程

**步骤**
- 对热水用户的历史数据进行选择性抽取，构建专家样本
- 数据探索和预处理，对用水样本数据建立用水事件时间间隔识别模型和划分一次完整用水事件模型，再在一次完整用水事件划分结果的基础上，剔除短暂用水事件，缩小识别范围
- 建立洗浴事件识别模型，对模型进行分析评价
- 应用模型，并对洗浴事件划分进行优化
- 调用洗浴事件识别模型，对实时监控的热水器流水数据进行洗浴事件自动识别

### 数据抽取

只能热水器在状态发生改变或者水流量非零时，每两秒会采集一条数据，数据总量非常巨大。

本案例对原始数据采用无放回随机抽样的方法抽取200家热水器用户从2014年1月1日到2014年12月31日的用水记录作为原始建模数据

### 数据探索分析

为了探索用户真实用水停顿时间间隔的分布情况，统计用户用水停顿时间并作频率分布直方图。

发现停顿时间间隔为0-0.3分钟的频率很高，为一次用水之间的停顿；停顿时间间隔为6-13分钟的频率很低，分析为两次用水之间的间隔。

### 数据预处理

**属性规约**
- 因为要对用户的一般规律进行挖掘分析，所以“热水器编号”可以去除；
- 热水器采集的数据中，“有无水流”可以用“水流量”反映，“有无水流”可以去除；
- “节能模式”都为“关”，可以去除；

**数值规约**
- 当热水器“开关机状态”为“关”且水流量为0时，说明热水器不处于工作状态，这些记录可以删除掉

**数据变换**

- 本案例的目标是对热水器用户的洗浴事件进行识别，这就需要从原始数据中识别出哪些状态是一个完整的用水事件，从而再识别出用水事件中的洗浴事件；
- 一次完整的用水事件是根据水流量和停顿时间间隔的阈值去划分的，所以本案例还建立了阈值寻优模型；
- 为了提高在大量的一次完整用水事件中寻找洗浴事件的效率，本案例建立了筛选规则提出可以明显判定不是洗浴的事件

In [10]:
# 阈值寻优
import numpy as np
import pandas as pd

inputfile = './data/data_10/water_heater.xls'
n = 4

threshold = pd.Timedelta(value=5, unit='m')
data = pd.read_excel(inputfile)
data['发生时间'] = data['发生时间'].astype(object)
data['发生时间'] = pd.to_datetime(data['发生时间'], format='%Y%m%d%H%M%S')
data = data[data['水流量'] > 0]

def even_num(ts):
    d = data['发生时间'].diff() > ts
    return d.sum() + 1

dt = [pd.Timedelta(value=i, unit='m') for i in np.arange(1, 9, 0.25)]
h = pd.DataFrame(dt, columns=['阈值'])
h['事件数'] = h['阈值'].apply(even_num)
h['斜率'] = h['事件数'].diff() / 0.25
h['斜率指标'] = h['斜率'].abs().rolling(n).mean()

ts = h['阈值'][h['斜率指标'].idxmin() - n]

if ts > threshold:
    ts = pd.Timedelta(value=5, unit='m')

print('最优时间间隔阈值为：{}'.format(ts))

最优时间间隔阈值为：0 days 00:04:00


In [11]:
# 划分用水事件
import pandas as pd

threshold = pd.Timedelta(value=4, unit='m')
inputfile = './data/data_10/water_heater.xls'
outputfile = './data/data_10/dividedsequence.xls'

data = pd.read_excel(inputfile)
data['发生时间'] = data['发生时间'].astype(object)
data['发生时间'] = pd.to_datetime(data['发生时间'], format='%Y%m%d%H%M%S')
data = data[data['水流量'] > 0]
data['事件编号'] = (data['发生时间'].diff() > threshold).cumsum() + 1

data.to_excel(outputfile)

In [12]:
data.head()

,发生时间,开关机状态,加热中,保温中,实际温度,热水量,水流量,加热剩余时间,当前设置温度,事件编号
2,2014-10-19 07:01:56,关,关,关,30°C,0%,8,0分钟,50°C,1
56,2014-10-19 07:38:16,关,关,关,30°C,0%,8,0分钟,50°C,2
381,2014-10-19 09:46:38,关,关,关,29°C,0%,16,0分钟,50°C,3
382,2014-10-19 09:46:40,关,关,关,29°C,0%,13,0分钟,50°C,3
384,2014-10-19 09:47:15,关,关,关,29°C,0%,20,0分钟,50°C,3


**属性构造**

本案例研究的是用水行为，构造以下4类指标：
- 时长指标：用水开始时间、用水结束时间、总用水时长、停顿时长、总停顿时长、用水时长、平均停顿时长、用水时长/总用水时长
- 频率指标：停顿次数
- 用水量化指标：总用水量、平均水流量
- 用水波动指标：水流量波动、停顿时长波动

**筛选得“候选洗浴事件”**
- 用三个比较宽松的条件筛选掉那些非常短暂的用水事件：
    - 一次用水事件中总用水量（纯热水）小于y升
    - 用水时长小于100秒
    - 总用水时长小于120秒

**数据清洗**

由于网络故障等原因，记录中存在延迟几十秒甚至几十分钟的情况，针对这种情况，如果使用平均值填充，会引入较大的误差，本案例的解决办法是在延迟记录的结束时间前加上一条记录，该记录水流量为0，时间在开始时间后加2秒。

### 模型构建

采用神经网络来进行洗浴事件的识别。

In [13]:
## 建立、训练多层神经网络，并完成模型的检验
np.random.seed(1)

inputfile1 = './data/data_10/train_neural_network_data.xls'
inputfile2 = './data/data_10/test_neural_network_data.xls'
testoutputfile = './data/data_10/test_output_data.xls'

data_train = pd.read_excel(inputfile1)
data_test = pd.read_excel(inputfile2)

y_train = data_train.iloc[:, 4].values
X_train = data_train.iloc[:, 5:16].values
y_test = data_test.iloc[:, 4].values
X_test = data_test.iloc[:, 5:16].values

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(17))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=0)
model.save_weights('./data/data_10/net.model')

r = pd.DataFrame(model.predict_classes(X_test), columns=['预测结果'])
r = pd.concat([data_test.iloc[:, :5], r], axis=1)
r.to_excel(testoutputfile)
r.head()

,热水事件,起始数据编号,终止数据编号,开始时间（begin_time）,根据日志判断是否为洗浴（1表示是，0表示否）,预测结果
0,1,73,336,2015-01-05 9:42:41',1,1
1,2,420,535,'2015-01-05 18:05:28',1,1
2,3,538,706,'2015-01-05 18:25:24',1,1
3,4,793,910,'2015-01-05 20:00:42',1,1
4,5,935,1133,'2015-01-05 20:15:13',1,1


In [14]:
accuracy = np.mean(r[u'根据日志判断是否为洗浴（1表示是，0表示否）'] == r['预测结果'])
print('模型在测试集上的准确率为：{:.2f}'.format(accuracy))

模型在测试集上的准确率为：0.86
